<a href="https://colab.research.google.com/github/aukkawut/News-Summarization/blob/main/T5_Fine_tuning_with_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Based on https://shivanandroy.com/fine-tune-t5-transformer-with-pytorch/

In [1]:
!pip install sentencepiece
!pip install transformers
!pip install rich[jupyter]

     |████████████████████████████████| 1.2 MB 4.1 MB/s 
     |████████████████████████████████| 3.4 MB 4.3 MB/s 
     |████████████████████████████████| 3.3 MB 50.4 MB/s 
     |████████████████████████████████| 61 kB 287 kB/s 
     |████████████████████████████████| 596 kB 54.6 MB/s 
     |████████████████████████████████| 895 kB 3.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 214 kB 4.1 MB/s 
     |████████████████████████████████| 51 kB 8.6 MB/s 


In [ ]:
df.sample(10)

In [ ]:
df["text"] = "summarize: "+df["text"]

In [ ]:
df.head()

In [ ]:
# Importing libraries
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import os

# Importing the T5 modules from huggingface/transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration

from rich.table import Column, Table
from rich import box
from rich.console import Console

# define a rich console logger
console=Console(record=True)

def display_df(df):
  """display dataframe in ASCII format"""

  console=Console()
  table = Table(Column("source_text", justify="center" ), Column("target_text", justify="center"), title="Sample Data",pad_edge=False, box=box.ASCII)

  for i, row in enumerate(df.values.tolist()):
    table.add_row(row[0], row[1])

  console.print(table)

training_logger = Table(Column("Epoch", justify="center" ), 
                        Column("Steps", justify="center"),
                        Column("Loss", justify="center"), 
                        title="Training Status",pad_edge=False, box=box.ASCII)


In [ ]:
# Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
class YourDataSetClass(Dataset):
  """
  Creating a custom dataset for reading the dataset and 
  loading it into the dataloader to pass it to the neural network for finetuning the model

  """

  def __init__(self, dataframe, tokenizer, source_len, target_len, source_text, target_text):
    self.tokenizer = tokenizer
    self.data = dataframe
    self.source_len = source_len
    self.summ_len = target_len
    self.target_text = self.data[target_text]
    self.source_text = self.data[source_text]

  def __len__(self):
    return len(self.target_text)

  def __getitem__(self, index):
    source_text = str(self.source_text[index])
    target_text = str(self.target_text[index])

    #cleaning data so as to ensure data is in string type
    source_text = ' '.join(source_text.split())
    target_text = ' '.join(target_text.split())

    source = self.tokenizer.batch_encode_plus([source_text], max_length= self.source_len, pad_to_max_length=True, truncation=True, padding="max_length", return_tensors='pt')
    target = self.tokenizer.batch_encode_plus([target_text], max_length= self.summ_len, pad_to_max_length=True, truncation=True, padding="max_length", return_tensors='pt')

    source_ids = source['input_ids'].squeeze()
    source_mask = source['attention_mask'].squeeze()
    target_ids = target['input_ids'].squeeze()
    target_mask = target['attention_mask'].squeeze()

    return {
        'source_ids': source_ids.to(dtype=torch.long), 
        'source_mask': source_mask.to(dtype=torch.long), 
        'target_ids': target_ids.to(dtype=torch.long),
        'target_ids_y': target_ids.to(dtype=torch.long)
    }

In [ ]:
def train(epoch, tokenizer, model, device, loader, optimizer):

  """
  Function to be called for training with the parameters passed from main function

  """

  model.train()
  for _,data in enumerate(loader, 0):
    y = data['target_ids'].to(device, dtype = torch.long)
    y_ids = y[:, :-1].contiguous()
    lm_labels = y[:, 1:].clone().detach()
    lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
    ids = data['source_ids'].to(device, dtype = torch.long)
    mask = data['source_mask'].to(device, dtype = torch.long)

    outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, labels=lm_labels)
    loss = outputs[0]

    if _%10==0:
      training_logger.add_row(str(epoch), str(_), str(loss))
      console.print(training_logger)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
def validate(epoch, tokenizer, model, device, loader):

  """
  Function to evaluate model for predictions

  """
  model.eval()
  predictions = []
  actuals = []
  with torch.no_grad():
      for _, data in enumerate(loader, 0):
          y = data['target_ids'].to(device, dtype = torch.long)
          ids = data['source_ids'].to(device, dtype = torch.long)
          mask = data['source_mask'].to(device, dtype = torch.long)

          generated_ids = model.generate(
              input_ids = ids,
              attention_mask = mask, 
              max_length=150, 
              num_beams=2,
              repetition_penalty=2.5, 
              length_penalty=1.0, 
              early_stopping=True
              )
          preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
          target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
          if _%10==0:
              console.print(f'Completed {_}')

          predictions.extend(preds)
          actuals.extend(target)
  return predictions, actuals

In [ ]:
def T5Trainer(dataframe, source_text, target_text, model_params, output_dir="./outputs/" ):
  
  """
  T5 trainer

  """

  # Set random seeds and deterministic pytorch for reproducibility
  torch.manual_seed(model_params["SEED"]) # pytorch random seed
  np.random.seed(model_params["SEED"]) # numpy random seed
  torch.backends.cudnn.deterministic = True

  # logging
  console.log(f"""[Model]: Loading {model_params["MODEL"]}...\n""")

  # tokenzier for encoding the text
  tokenizer = T5Tokenizer.from_pretrained(model_params["MODEL"])

  # Defining the model. We are using t5-base model and added a Language model layer on top for generation of Summary. 
  # Further this model is sent to device (GPU/TPU) for using the hardware.
  model = T5ForConditionalGeneration.from_pretrained(model_params["MODEL"])
  model = model.to(device)
  
  # logging
  console.log(f"[Data]: Reading data...\n")

  # Importing the raw dataset
  dataframe = dataframe[[source_text,target_text]]
  display_df(dataframe.head(2))

  
  # Creation of Dataset and Dataloader
  # Defining the train size. So 80% of the data will be used for training and the rest for validation. 
  train_size = 0.8
  train_dataset=dataframe.sample(frac=train_size,random_state = model_params["SEED"])
  val_dataset=dataframe.drop(train_dataset.index).reset_index(drop=True)
  train_dataset = train_dataset.reset_index(drop=True)

  console.print(f"FULL Dataset: {dataframe.shape}")
  console.print(f"TRAIN Dataset: {train_dataset.shape}")
  console.print(f"TEST Dataset: {val_dataset.shape}\n")


  # Creating the Training and Validation dataset for further creation of Dataloader
  training_set = YourDataSetClass(train_dataset, tokenizer, model_params["MAX_SOURCE_TEXT_LENGTH"], model_params["MAX_TARGET_TEXT_LENGTH"], source_text, target_text)
  val_set = YourDataSetClass(val_dataset, tokenizer, model_params["MAX_SOURCE_TEXT_LENGTH"], model_params["MAX_TARGET_TEXT_LENGTH"], source_text, target_text)


  # Defining the parameters for creation of dataloaders
  train_params = {
      'batch_size': model_params["TRAIN_BATCH_SIZE"],
      'shuffle': True,
      'num_workers': 0
      }


  val_params = {
      'batch_size': model_params["VALID_BATCH_SIZE"],
      'shuffle': False,
      'num_workers': 0
      }


  # Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
  training_loader = DataLoader(training_set, **train_params)
  val_loader = DataLoader(val_set, **val_params)


  # Defining the optimizer that will be used to tune the weights of the network in the training session. 
  optimizer = torch.optim.Adam(params =  model.parameters(), lr=model_params["LEARNING_RATE"])


  # Training loop
  console.log(f'[Initiating Fine Tuning]...\n')

  for epoch in range(model_params["TRAIN_EPOCHS"]):
      train(epoch, tokenizer, model, device, training_loader, optimizer)
      
  console.log(f"[Saving Model]...\n")
  #Saving the model after training
  path = os.path.join(output_dir, "model_files")
  model.save_pretrained(path)
  tokenizer.save_pretrained(path)


  # evaluating test dataset
  console.log(f"[Initiating Validation]...\n")
  for epoch in range(model_params["VAL_EPOCHS"]):
    predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
    final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})
    final_df.to_csv(os.path.join(output_dir,'predictions.csv'))
  
  console.save_text(os.path.join(output_dir,'logs.txt'))
  
  console.log(f"[Validation Completed.]\n")
  console.print(f"""[Model] Model saved @ {os.path.join(output_dir, "model_files")}\n""")
  console.print(f"""[Validation] Generation on Validation data saved @ {os.path.join(output_dir,'predictions.csv')}\n""")
  console.print(f"""[Logs] Logs saved @ {os.path.join(output_dir,'logs.txt')}\n""")

In [ ]:
model_params={
    "MODEL":"t5-base",             # model_type: t5-base/t5-large
    "TRAIN_BATCH_SIZE":8,          # training batch size
    "VALID_BATCH_SIZE":8,          # validation batch size
    "TRAIN_EPOCHS":3,              # number of training epochs
    "VAL_EPOCHS":1,                # number of validation epochs
    "LEARNING_RATE":1e-4,          # learning rate
    "MAX_SOURCE_TEXT_LENGTH":512,  # max length of source text
    "MAX_TARGET_TEXT_LENGTH":50,   # max length of target text
    "SEED": 42                     # set seed for reproducibility 

}

In [ ]:
T5Trainer(dataframe=df, source_text="text", target_text="headlines", model_params=model_params, output_dir="outputs")

This part is the part I messed up as I forgot that I'm working on the notebook that I did not own -> no save for me. So, the code on PEGASUS and GAN is lost. I tried to recreate that as close as possible but please understand that I might not test it. I tried my best to dig down into the runtime memory and this is what I recovered for GAN. The PEGASUS one can be done by changing the tokenizer and the model on T5Trainer function. Sadly, that part of the code was purged out from the memory before I can recover it (as I did that long time ago, compared to gan)

In [1]:
!pip install sentencepiece
!pip install transformers
!pip install rich[jupyter]
!wget http://aukkawut.github.io/NYT_dataset.zip
!unzip NYT_dataset.zip

     |████████████████████████████████| 1.2 MB 6.8 MB/s 
     |████████████████████████████████| 3.4 MB 8.3 MB/s 
     |████████████████████████████████| 895 kB 41.9 MB/s 
     |████████████████████████████████| 3.3 MB 33.7 MB/s 
     |████████████████████████████████| 61 kB 567 kB/s 
     |████████████████████████████████| 596 kB 40.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 214 kB 8.4 MB/s 
     |████████████████████████████████| 51 kB 7.3 MB/s 
--2021-12-17 03:22:07--  http://aukkawut.github.io/NYT_dataset.zip
Resolving aukkawut.github.io (aukkawut.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to aukkawut.github.io (aukkawut.github.io)|185.199.108.153|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://aukkawut.info/NYT_dataset.zip [following]
--2021-12

In [2]:
import pandas as pd
df = pd.read_csv('NYT_Dataset.csv',index_col=0)[['abstract','title']]
df["abstract"] = "summarize: "+df["abstract"]

In [3]:
# Importing libraries
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import os

# Importing the T5 modules from huggingface/transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration, PegasusTokenizer, PegasusForConditionalGeneration, BertTokenizerFast, BertForSequenceClassification

from rich.table import Column, Table
from rich import box
from rich.console import Console

# define a rich console logger
console=Console(record=True)

def display_df(df):
  """display dataframe in ASCII format"""

  console=Console()
  table = Table(Column("source_text", justify="center" ), Column("target_text", justify="center"), title="Sample Data",pad_edge=False, box=box.ASCII)

  for i, row in enumerate(df.values.tolist()):
    table.add_row(row[0], row[1])

  console.print(table)

training_logger = Table(Column("Epoch", justify="center" ), 
                        Column("Steps", justify="center"),
                        Column("Loss", justify="center"), 
                        title="Training Status",pad_edge=False, box=box.ASCII)
gtraining_logger = Table(Column("Epoch", justify="center" ), 
                        Column("Steps", justify="center"),
                        Column("GLoss", justify="center"), 
                        Column("DLoss", justify="center"),
                        title="Training Status",pad_edge=False, box=box.ASCII)
# Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
class YourDataSetClass(Dataset):
  """
  Creating a custom dataset for reading the dataset and 
  loading it into the dataloader to pass it to the neural network for finetuning the model

  """

  def __init__(self, dataframe, tokenizer, source_len, target_len, source_text, target_text):
    self.tokenizer = tokenizer
    self.data = dataframe
    self.source_len = source_len
    self.summ_len = target_len
    self.target_text = self.data[target_text]
    self.source_text = self.data[source_text]

  def __len__(self):
    return len(self.target_text)

  def __getitem__(self, index):
    source_text = str(self.source_text[index])
    target_text = str(self.target_text[index])

    #cleaning data so as to ensure data is in string type
    source_text = ' '.join(source_text.split())
    target_text = ' '.join(target_text.split())

    source = self.tokenizer.batch_encode_plus([source_text], max_length= self.source_len, pad_to_max_length=True, truncation=True, padding="max_length", return_tensors='pt')
    target = self.tokenizer.batch_encode_plus([target_text], max_length= self.summ_len, pad_to_max_length=True, truncation=True, padding="max_length", return_tensors='pt')

    source_ids = source['input_ids'].squeeze()
    source_mask = source['attention_mask'].squeeze()
    target_ids = target['input_ids'].squeeze()
    target_mask = target['attention_mask'].squeeze()

    return {
        'source_ids': source_ids.to(dtype=torch.long), 
        'source_mask': source_mask.to(dtype=torch.long), 
        'target_ids': target_ids.to(dtype=torch.long),
        'target_ids_y': target_ids.to(dtype=torch.long)
    }
def train(epoch, tokenizer, model, device, loader, optimizer):

  """
  Function to be called for training with the parameters passed from main function

  """

  model.train()
  for _,data in enumerate(loader, 0):
    y = data['target_ids'].to(device, dtype = torch.long)
    y_ids = y[:, :-1].contiguous()
    lm_labels = y[:, 1:].clone().detach()
    lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
    ids = data['source_ids'].to(device, dtype = torch.long)
    mask = data['source_mask'].to(device, dtype = torch.long)

    outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, labels=lm_labels)
    loss = outputs[0]

    if _%10==0:
      training_logger.add_row(str(epoch), str(_), str(loss))
      console.print(training_logger)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
def validate(epoch, tokenizer, model, device, loader):

  """
  Function to evaluate model for predictions

  """
  model.eval()
  predictions = []
  actuals = []
  with torch.no_grad():
      for _, data in enumerate(loader, 0):
          y = data['target_ids'].to(device, dtype = torch.long)
          ids = data['source_ids'].to(device, dtype = torch.long)
          mask = data['source_mask'].to(device, dtype = torch.long)

          generated_ids = model.generate(
              input_ids = ids,
              attention_mask = mask, 
              max_length=150, 
              num_beams=2,
              repetition_penalty=2.5, 
              length_penalty=1.0, 
              early_stopping=True
              )
          preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
          target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
          if _%10==0:
              console.print(f'Completed {_}')

          predictions.extend(preds)
          actuals.extend(target)
  return predictions, actuals
criterion = torch.nn.BCEWithLogitsLoss()
def ganT5_Trainer(dataframe, source_text, target_text, model_params, output_dir="./outputs/"):
  
  """
  T5 trainer

  """

  # Set random seeds and deterministic pytorch for reproducibility
  torch.manual_seed(model_params["SEED"]) # pytorch random seed
  np.random.seed(model_params["SEED"]) # numpy random seed
  torch.backends.cudnn.deterministic = True

  # logging
  console.log(f"""[Model]: Loading {model_params["MODEL"]}...\n""")

  # tokenzier for encoding the text
  tokenizer = T5Tokenizer.from_pretrained(model_params["MODEL"])
  btokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
  # Defining the model. We are using t5-base model and added a Language model layer on top for generation of Summary. 
  # Further this model is sent to device (GPU/TPU) for using the hardware.
  model = T5ForConditionalGeneration.from_pretrained(model_params["MODEL"])
  model = model.to(device)
  
  # logging
  console.log(f"[Data]: Reading data...\n")

  # Importing the raw dataset
  dataframe = dataframe[[source_text,target_text]]
  display_df(dataframe.head(2))

  
  # Creation of Dataset and Dataloader
  # Defining the train size. So 80% of the data will be used for training and the rest for validation. 
  train_size = 0.8
  train_dataset=dataframe.sample(frac=train_size,random_state = model_params["SEED"])
  val_dataset=dataframe.drop(train_dataset.index).reset_index(drop=True)
  train_dataset = train_dataset.reset_index(drop=True)

  console.print(f"FULL Dataset: {dataframe.shape}")
  console.print(f"TRAIN Dataset: {train_dataset.shape}")
  console.print(f"TEST Dataset: {val_dataset.shape}\n")


  # Creating the Training and Validation dataset for further creation of Dataloader
  training_set = YourDataSetClass(train_dataset, tokenizer, model_params["MAX_SOURCE_TEXT_LENGTH"], model_params["MAX_TARGET_TEXT_LENGTH"], source_text, target_text)
  val_set = YourDataSetClass(val_dataset, tokenizer, model_params["MAX_SOURCE_TEXT_LENGTH"], model_params["MAX_TARGET_TEXT_LENGTH"], source_text, target_text)
  btraining_set = YourDataSetClass(train_dataset, btokenizer, model_params["MAX_SOURCE_TEXT_LENGTH"], model_params["MAX_TARGET_TEXT_LENGTH"], source_text, target_text)
  bval_set = YourDataSetClass(val_dataset, btokenizer, model_params["MAX_SOURCE_TEXT_LENGTH"], model_params["MAX_TARGET_TEXT_LENGTH"], source_text, target_text)

  # Defining the parameters for creation of dataloaders
  train_params = {
      'batch_size': model_params["TRAIN_BATCH_SIZE"],
      'shuffle': True,
      'num_workers': 0
      }


  val_params = {
      'batch_size': model_params["VALID_BATCH_SIZE"],
      'shuffle': False,
      'num_workers': 0
      }


  # Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
  training_loader = DataLoader(training_set, **train_params)
  val_loader = DataLoader(val_set, **val_params)
  btraining_loader = DataLoader(btraining_set, **train_params)
  bval_loader = DataLoader(bval_set, **val_params)

  # Defining the optimizer that will be used to tune the weights of the network in the training session. 
  optimizer = torch.optim.Adam(params =  model.parameters(), lr=model_params["LEARNING_RATE"])
  doptimizer = torch.optim.Adam(params =  model.parameters(), lr=model_params["LEARNING_RATE"])


  #prepare discriminator
  console.log(f'[Initiating GAN Fine Tuning]...\n')
 
  discriminator = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2).cuda()
  discriminator = discriminator.to(device)
  # Training loop
  D_labels = torch.ones(model_params["TRAIN_BATCH_SIZE"], 2).to(device) # Discriminator Label to real
  D_fakes = torch.zeros(model_params["TRAIN_BATCH_SIZE"], 2).to(device) # Discriminator Label to fake
  for epoch in range(model_params["TRAIN_EPOCHS"]):
      '''
      TODO
      '''
      console.log(f"""[Epoch :{epoch}/{model_params["TRAIN_EPOCHS"]}]\n""")
      for _,(data1,data2) in enumerate(zip(btraining_loader,training_loader), 0):
          #train discriminator: BERT model
          #real data
          y = data1['target_ids'].to(device, dtype = torch.long)
          b_label = D_labels
          ids = data1['source_ids'].to(device, dtype = torch.long)
          mask = data1['source_mask'].to(device, dtype = torch.long)

          outputs = discriminator(input_ids = ids, attention_mask = mask, labels=b_label)
          loss = outputs[0]
          D_x_loss = criterion(outputs.logits.float(), D_labels)
          #fake data
          y = data2['target_ids'].to(device, dtype = torch.long)
          y_ids = y[:, :-1].contiguous()
          lm_labels = y[:, 1:].clone().detach()
          lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
          ids = data2['source_ids'].to(device, dtype = torch.long)
          mask = data2['source_mask'].to(device, dtype = torch.long)
          fake_gen = tokenizer.decode(model.generate(ids)[0], skip_special_tokens=True)
          fake_end = btokenizer([fake_gen], max_length= 512, pad_to_max_length=True, truncation=True, padding="max_length", return_tensors='pt').to(device)
          source_ids = fake_end['input_ids'].squeeze().to(device, dtype = torch.long)
          source_mask = fake_end['attention_mask'].squeeze().to(device, dtype = torch.long)
          outputs = discriminator(**fake_end, labels=D_fakes)
          D_z_loss = criterion(outputs.logits.float(), D_fakes)
          D_loss = D_x_loss + D_z_loss
        
          discriminator.zero_grad()
          D_loss.backward()
          doptimizer.step()
      #train generator: T5 model
          fake_gen = tokenizer.decode(model.generate(ids)[0], skip_special_tokens=True)
          fake_end = btokenizer.batch_encode_plus([fake_gen], max_length= 512, pad_to_max_length=True, truncation=True, padding="max_length", return_tensors='pt').to(device)
          source_ids = fake_end['input_ids'].squeeze().to(device, dtype = torch.long)
          source_mask = fake_end['attention_mask'].squeeze().to(device, dtype = torch.long)
          outputs2 = discriminator(**fake_end)
          G_loss = criterion(outputs2.logits.float(), D_labels)
          model.zero_grad()
          G_loss.backward()
          optimizer.step()
          if _%100==0:
            gtraining_logger.add_row(str(epoch), str(_), str(D_loss.item()), str(G_loss.item()))
            console.print(gtraining_logger)
          
  console.log(f"[Saving Model]...\n")
  #Saving the model after training
  path = os.path.join(output_dir, "model_files")
  model.save_pretrained(path)
  tokenizer.save_pretrained(path)


  # evaluating test dataset
  console.log(f"[Initiating Validation]...\n")
  for epoch in range(model_params["VAL_EPOCHS"]):
    predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
    final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})
    final_df.to_csv(os.path.join(output_dir,'predictions.csv'))
  
  console.save_text(os.path.join(output_dir,'logs.txt'))
  
  console.log(f"[Validation Completed.]\n")
  console.print(f"""[Model] Model saved @ {os.path.join(output_dir, "model_files")}\n""")
  console.print(f"""[Validation] Generation on Validation data saved @ {os.path.join(output_dir,'predictions.csv')}\n""")
  console.print(f"""[Logs] Logs saved @ {os.path.join(output_dir,'logs.txt')}\n""")

model_params={
    "MODEL":"t5-base",             # model_type: t5-base/t5-large
    "TRAIN_BATCH_SIZE":1,          # training batch size
    "VALID_BATCH_SIZE":1,          # validation batch size
    "TRAIN_EPOCHS":10,              # number of training epochs
    "VAL_EPOCHS":1,                # number of validation epochs
    "LEARNING_RATE":1e-4,          # learning rate
    "MAX_SOURCE_TEXT_LENGTH":512,  # max length of source text
    "MAX_TARGET_TEXT_LENGTH":50,   # max length of target text
    "SEED": 42                     # set seed for reproducibility 

}


In [ ]:
ganT5_Trainer(dataframe=df, source_text="abstract", target_text="title", model_params=model_params, output_dir="outputs")

In [6]:
import pandas as pd

In [ ]:
t5 = pd.read_csv('T5.csv', index_col=0)
gant5 = pd.read_csv('ganT5.csv', index_col=0)
pegasus = pd.read_csv('pegasus.csv', index_col=0)

ValueError: ignored

In [ ]:
for x in pegasus['Actual Text']:
  for y in gant5['Actual Text']:
    if x == y:
      print(x)

Saudi Arabia: Executions Rose in 2007


In [ ]:
gant5[gant5['Actual Text'] == "Iraqi Revelers Embrace the New Year"].values[0,0]

IndexError: ignored

In [ ]:
pegasus[pegasus['Actual Text'] == "Saudi Arabia: Executions Rose in 2007"].values[0,0]

'Executions Increase in Saudi Arabia, Report Finds'

In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [5]:
!cp ./drive/MyDrive/t5FineTune.zip t5.zip
!unzip t5.zip

Archive:  t5.zip
   creating: outputs_t5/
  inflating: outputs_t5/predictions.csv  
  inflating: outputs_t5/logs.txt     
   creating: outputs_t5/model_files/
  inflating: outputs_t5/model_files/config.json  
  inflating: outputs_t5/model_files/pytorch_model.bin  
  inflating: outputs_t5/model_files/spiece.model  
  inflating: outputs_t5/model_files/tokenizer_config.json  
  inflating: outputs_t5/model_files/special_tokens_map.json  


In [6]:
!cp ./drive/MyDrive/ganT5.zip gant5.zip
!unzip gant5.zip

Archive:  gant5.zip
   creating: outputs_ganT5/
  inflating: outputs_ganT5/predictions.csv  
  inflating: outputs_ganT5/logs.txt  
   creating: outputs_ganT5/model_files/
  inflating: outputs_ganT5/model_files/config.json  
  inflating: outputs_ganT5/model_files/pytorch_model.bin  
  inflating: outputs_ganT5/model_files/spiece.model  
  inflating: outputs_ganT5/model_files/tokenizer_config.json  
  inflating: outputs_ganT5/model_files/special_tokens_map.json  


In [7]:
!cp ./drive/MyDrive/PegasusFineTune.zip peg.zip
!unzip peg.zip

Archive:  peg.zip
   creating: outputs_peg/
  inflating: outputs_peg/predictions.csv  
  inflating: outputs_peg/logs.txt    
   creating: outputs_peg/model_files/
  inflating: outputs_peg/model_files/config.json  
  inflating: outputs_peg/model_files/pytorch_model.bin  
  inflating: outputs_peg/model_files/spiece.model  
  inflating: outputs_peg/model_files/tokenizer_config.json  
  inflating: outputs_peg/model_files/special_tokens_map.json  


In [8]:
from transformers import T5Config

In [9]:
output_model_file = "./outputs_t5/model_files/pytorch_model.bin"
output_config_file = "./outputs_t5/model_files/config.json"
config = T5Config.from_json_file(output_config_file)
ft5 = T5ForConditionalGeneration(config)
state_dict = torch.load(output_model_file)
ft5.load_state_dict(state_dict)

<All keys matched successfully>

In [10]:
t5 = T5ForConditionalGeneration.from_pretrained("t5-base")

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

In [11]:
goutput_model_file = "./outputs_ganT5/model_files/pytorch_model.bin"
goutput_config_file = "./outputs_ganT5/model_files/config.json"
gconfig = T5Config.from_json_file(goutput_config_file)
gt5 = T5ForConditionalGeneration(gconfig)
gstate_dict = torch.load(goutput_model_file)
gt5.load_state_dict(gstate_dict)

<All keys matched successfully>

In [12]:
tokenizer = T5Tokenizer.from_pretrained("t5-base")

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

In [13]:
eval = df.dropna().sample(n=50)

In [14]:
eval

,abstract,title
23369,summarize: A coal ash pond ruptured and sent a...,Toxic Coal Ash Spill in Tennessee
23329,summarize: The European Union’s highest court ...,Ruling Favors a 10-Inch Citizen of France
10944,summarize: Ann Curry of NBC News talks with Pr...,Interview with President Ahmadinejad
42672,summarize: The army dismissed charges against ...,India Dismisses Charges Against Soldiers in Ki...
86748,summarize: Jair Bolsonaro’s view is not mainst...,Brazil’s President Tells Armed Forces to Comme...
74534,"summarize: In Syria, Nigeria and elsewhere, Pr...",Trump Faces the Limits of American Power
29717,"summarize: Faced with high shipping costs, Zim...","Using U.S. Dollars, Zimbabwe Finds a Problem: ..."
52615,"summarize: Prince Mohammed bin Nayef, the new ...",Saudi Succession Hints at Shift in Foreign Role
30949,summarize: Members of United Nations agencies ...,Aid Workers Detained in Myanmar
62140,summarize: From the International Herald Tribu...,1915: War Hospital In Paris Becomes Prey To Fl...


In [15]:
t = tokenizer(eval['abstract'].to_list(), padding=True, truncation=True,return_tensors="pt").input_ids

In [16]:
o1 = t5.generate(t)

In [17]:
text1 = []
for x in o1:
  text1.append(tokenizer.decode(x, skip_special_tokens=True))

In [18]:
o2 = ft5.generate(t)
o3 = gt5.generate(t)

In [19]:
text2 = []
text3 = []
for x in o2:
  text2.append(tokenizer.decode(x, skip_special_tokens=True))
for x in o3:
  text3.append(tokenizer.decode(x, skip_special_tokens=True))

In [20]:
eval['t5'] = text1
eval['ft5'] = text2
eval['gt5'] = text3

In [31]:
eval.to_csv('eval.csv')

In [23]:
from transformers import PegasusConfig

In [24]:
poutput_model_file = "./outputs_peg/model_files/pytorch_model.bin"
poutput_config_file = "./outputs_peg/model_files/config.json"
pconfig = PegasusConfig.from_json_file(poutput_config_file)
fpeg = PegasusForConditionalGeneration(pconfig)
pstate_dict = torch.load(poutput_model_file)
fpeg.load_state_dict(pstate_dict)

<All keys matched successfully>

In [25]:
peg = PegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")

Downloading:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

In [26]:
ptokenizer = PegasusTokenizer.from_pretrained("google/pegasus-xsum")

Downloading:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.36M [00:00<?, ?B/s]

In [27]:
t = ptokenizer(eval['abstract'].to_list(), padding=True, truncation=True,return_tensors="pt").input_ids

In [28]:
o2 = peg.generate(t)
o3 = fpeg.generate(t)

In [29]:
text2 = []
text3 = []
for x in o2:
  text2.append(ptokenizer.decode(x, skip_special_tokens=True))
for x in o3:
  text3.append(ptokenizer.decode(x, skip_special_tokens=True))

In [30]:
eval['peg'] = text2
eval['fpeg'] = text3

In [ ]:
target = ptokenizer(s, return_tensors="pt").input_ids
o = pmodel.generate(target)
print(ptokenizer.decode(o[0], skip_special_tokens=True))

Problem Is Not Easy to Solve


In [32]:
!pip install rouge

In [33]:
from rouge import Rouge

In [34]:
rouge = Rouge()

In [64]:
eval = eval.replace('hello', 'pfdoapskofjei')

In [65]:
eval.iloc[2].fpeg

'pfdoapskofjei'

In [67]:
rouge.get_scores(eval.peg.to_list(), eval.abstract.to_list(),avg=True)

{'rouge-1': {'f': 0.2670509152606704,
  'p': 0.31582684297201724,
  'r': 0.2395367369897005},
 'rouge-2': {'f': 0.12849068041461498,
  'p': 0.14365656786043113,
  'r': 0.11845589144529395},
 'rouge-l': {'f': 0.2392432684721754,
  'p': 0.2828460688441843,
  'r': 0.21509017215774545}}

In [ ]:
po="a police officer in Russia uses the Internet to challenge the country's leaders."
s = df[df["title"]=="Iraqi Revelers Embrace the New Year"].values[0,0]

In [ ]:
s

"summarize: A police officer in Russia uses the Internet to challenge the country's leaders to address rampant corruption."

In [ ]:
rouge.get_scores(po,"A police officer in Russia uses the Internet to challenge the country's leaders to address rampant corruption.")

[{'rouge-1': {'f': 0.8148148098765432,
   'p': 0.9166666666666666,
   'r': 0.7333333333333333},
  'rouge-2': {'f': 0.7857142808163265, 'p': 0.9166666666666666, 'r': 0.6875},
  'rouge-l': {'f': 0.8148148098765432,
   'p': 0.9166666666666666,
   'r': 0.7333333333333333}}]